# Transformer

<div class="alert alert-info">

This tutorial is available as an IPython notebook at [Malaya/example/transformer](https://github.com/huseinzol05/Malaya/tree/master/example/transformer).
    
</div>

Below are the list of dataset we pretrained,

Standard Bahasa dataset, 

1. [Malay-dataset/dumping](https://github.com/huseinzol05/Malay-Dataset/tree/master/dumping).
2. [Malay-dataset/pure-text](https://github.com/huseinzol05/Malay-Dataset/tree/master/pure-text).

Bahasa social media,

1. [Malay-dataset/dumping/instagram](https://github.com/huseinzol05/Malay-Dataset/tree/master/dumping/instagram).
2. [Malay-dataset/dumping/twitter](https://github.com/huseinzol05/Malay-Dataset/tree/master/dumping/twitter).

Singlish / Manglish,

1. [Malay-dataset/dumping/singlish](https://github.com/huseinzol05/Malay-Dataset/tree/master/dumping/singlish-text).
2. [Malay-dataset/dumping/singapore-news](https://github.com/huseinzol05/Malay-Dataset/tree/master/dumping/singapore-news).

For T5 transformer models, the models trained on MNLI both english and malay languages.

**This interface not able us to use it to do custom training**.

If you want to download pretrained model for Transformer-Bahasa and use it for custom transfer-learning, you can download it here, https://github.com/huseinzol05/Malaya/tree/master/pretrained-model/, some notebooks to help you get started.

In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
%%time
import malaya

/home/husein/.local/lib/python3.8/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/home/husein/.local/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CPU times: user 2.72 s, sys: 2.27 s, total: 4.99 s
Wall time: 2.51 s


/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


### list Transformer HuggingFace available

In [3]:
malaya.transformer.available_huggingface

{'mesolitica/roberta-base-bahasa-cased': {'Size (MB)': 443},
 'mesolitica/roberta-tiny-bahasa-cased': {'Size (MB)': 66.1},
 'mesolitica/bert-base-standard-bahasa-cased': {'Size (MB)': 443},
 'mesolitica/bert-tiny-standard-bahasa-cased': {'Size (MB)': 66.1},
 'mesolitica/roberta-base-standard-bahasa-cased': {'Size (MB)': 443},
 'mesolitica/roberta-tiny-standard-bahasa-cased': {'Size (MB)': 66.1},
 'mesolitica/electra-base-generator-bahasa-cased': {'Size (MB)': 140},
 'mesolitica/electra-small-generator-bahasa-cased': {'Size (MB)': 19.3}}

In [4]:
strings = ['Kerajaan galakkan rakyat naik public transport tapi parking kat lrt ada 15. Reserved utk staff rapid je dah berpuluh. Park kereta tepi jalan kang kene saman dgn majlis perbandaran. Kereta pulak senang kene curi. Cctv pun tak ada. Naik grab dah 5-10 ringgit tiap hari. Gampang juga',
           'Alaa Tun lek ahhh npe muka masam cmni kn agong kata usaha kerajaan terdahulu sejak selepas merdeka',
           "Orang ramai cakap nurse kerajaan garang. So i tell u this. Most of our local ppl will treat us as hamba abdi and they don't respect us as a nurse",
          'Pemuda mogok lapar desak kerajaan prihatin isu iklim',
          'kerajaan perlu kisah isu iklim, pemuda mogok lapar',
          'Kerajaan dicadang tubuh jawatankuasa khas tangani isu alam sekitar']

### Load HuggingFace model

```python
def huggingface(
    model: str = 'mesolitica/electra-base-generator-bahasa-cased',
    force_check: bool = True,
    **kwargs,
):
    """
    Load transformer model.

    Parameters
    ----------
    model: str, optional (default='mesolitica/electra-base-generator-bahasa-cased')
        Check available models at `malaya.transformer.available_transformer()`.
    force_check: bool, optional (default=True)
        Force check model one of malaya model.
        Set to False if you have your own huggingface model.
    """
```

In [5]:
model = malaya.transformer.huggingface(model = 'mesolitica/electra-base-generator-bahasa-cased')

Loading the tokenizer from the `special_tokens_map.json` and the `added_tokens.json` will be removed in `transformers 5`,  it is kept for forward compatibility, but it is recommended to update your `tokenizer_config.json` by uploading it again. You will see the new `added_tokens_decoder` attribute that will store the relevant information.


I have random sentences copied from Twitter, searched using `kerajaan` keyword.

#### Vectorization

Change a string or batch of strings to latent space / vectors representation.

```python
def vectorize(
    self,
    strings: List[str],
    method: str = 'last',
    method_token: str = 'first',
    t5_head_logits: bool = True,
    **kwargs,
):
    """
    Vectorize string inputs.

    Parameters
    ----------
    strings: List[str]
    method: str, optional (default='last')
        hidden layers supported. Allowed values:

        * ``'last'`` - last layer.
        * ``'first'`` - first layer.
        * ``'mean'`` - average all layers.

        This only applicable for non T5 models.
    method_token: str, optional (default='first')
        token layers supported. Allowed values:

        * ``'last'`` - last token.
        * ``'first'`` - first token.
        * ``'mean'`` - average all tokens.

        usually pretrained models trained on `first` token for classification task.
        This only applicable for non T5 models.
    t5_head_logits: str, optional (default=True)
        if True, will take head logits, else, last token.
        This only applicable for T5 models.

    Returns
    -------
    result: np.array
    """
```

In [6]:
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
v = model.vectorize(strings)
v.shape

You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


(6, 256)

In [8]:
cosine_similarity(v)

array([[1.0000001 , 0.72213906, 0.70548344, 0.6682126 , 0.6442658 ,
        0.68018407],
       [0.72213906, 1.        , 0.62266254, 0.7186686 , 0.69928503,
        0.7106037 ],
       [0.70548344, 0.62266254, 0.9999999 , 0.6309348 , 0.63519955,
        0.62969273],
       [0.6682126 , 0.7186686 , 0.6309348 , 1.0000001 , 0.9547028 ,
        0.8564713 ],
       [0.6442658 , 0.69928503, 0.63519955, 0.9547028 , 1.0000001 ,
        0.8234203 ],
       [0.68018407, 0.7106037 , 0.62969273, 0.8564713 , 0.8234203 ,
        1.        ]], dtype=float32)

#### Attention

```python
def attention(
    self,
    strings: List[str],
    method: str = 'last',
    method_head: str = 'mean',
    t5_attention: str = 'cross_attentions',
    **kwargs,
):
    """
    Get attention string inputs.

    Parameters
    ----------
    strings: List[str]
    method: str, optional (default='last')
        Attention layer supported. Allowed values:

        * ``'last'`` - attention from last layer.
        * ``'first'`` - attention from first layer.
        * ``'mean'`` - average attentions from all layers.
    method_head: str, optional (default='mean')
        attention head layer supported. Allowed values:

        * ``'last'`` - attention from last layer.
        * ``'first'`` - attention from first layer.
        * ``'mean'`` - average attentions from all layers.
    t5_attention: str, optional (default='cross_attentions')
        attention type for T5 models. Allowed values:

        * ``'cross_attentions'`` - cross attention.
        * ``'encoder_attentions'`` - encoder attention.
        * ``'decoder_attentions'`` - decoder attention.

        This only applicable for T5 models.

    Returns
    -------
    result : List[List[Tuple[str, float]]]
    """
```

You can give list of strings or a string to get the attention, in this documentation, I just want to use a string.

In [11]:
model.attention([strings[1]], method = 'last')

[[('Alaa', 0.05886863),
  ('Tun', 0.0612526),
  ('lek', 0.06898941),
  ('ahhh', 0.064397976),
  ('npe', 0.050825186),
  ('muka', 0.07244484),
  ('masam', 0.053202268),
  ('cmni', 0.04823282),
  ('kn', 0.058162007),
  ('agong', 0.06559846),
  ('kata', 0.05514032),
  ('usaha', 0.057437427),
  ('kerajaan', 0.04105992),
  ('terdahulu', 0.044371355),
  ('sejak', 0.06925423),
  ('selepas', 0.06948459),
  ('merdeka', 0.061277937)]]

In [12]:
model.attention([strings[1]], method = 'first')

[[('Alaa', 0.061838076),
  ('Tun', 0.053071998),
  ('lek', 0.047781985),
  ('ahhh', 0.046944533),
  ('npe', 0.052150372),
  ('muka', 0.05392791),
  ('masam', 0.058074415),
  ('cmni', 0.08068735),
  ('kn', 0.05034356),
  ('agong', 0.054398913),
  ('kata', 0.057019003),
  ('usaha', 0.058209922),
  ('kerajaan', 0.06937862),
  ('terdahulu', 0.080670245),
  ('sejak', 0.05798509),
  ('selepas', 0.06437355),
  ('merdeka', 0.05314437)]]

In [13]:
model.attention([strings[1]], method = 'mean')

[[('Alaa', 0.048754197),
  ('Tun', 0.054038025),
  ('lek', 0.053129513),
  ('ahhh', 0.057060346),
  ('npe', 0.04947073),
  ('muka', 0.06097326),
  ('masam', 0.057632357),
  ('cmni', 0.07236169),
  ('kn', 0.052900266),
  ('agong', 0.0538029),
  ('kata', 0.07015142),
  ('usaha', 0.061375342),
  ('kerajaan', 0.06380817),
  ('terdahulu', 0.063899584),
  ('sejak', 0.05665373),
  ('selepas', 0.0524459),
  ('merdeka', 0.07154254)]]

In [15]:
roberta = malaya.transformer.huggingface(model = 'mesolitica/roberta-base-standard-bahasa-cased')

In [16]:
roberta.attention([strings[1]], method = 'last')

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


[[('Alaa', 0.052424464),
  ('Tun', 0.08523697),
  ('lek', 0.06813958),
  ('ahhh', 0.06153967),
  ('npe', 0.06513652),
  ('muka', 0.059199475),
  ('masam', 0.061626352),
  ('cmni', 0.067371994),
  ('kn', 0.0662273),
  ('agong', 0.052743725),
  ('kata', 0.06723866),
  ('usaha', 0.04410252),
  ('kerajaan', 0.060376044),
  ('terdahulu', 0.04183174),
  ('sejak', 0.04189242),
  ('selepas', 0.039302673),
  ('merdeka', 0.06560987)]]